In [17]:
import matplotlib.pyplot as plt

import numpy as np
import scipy.io as sio
from scipy.stats import ortho_group
from sklearn.svm import SVC 
import math 
import random
from numpy import linalg as LA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn.linear_model import orthogonal_mp

import time

In [2]:
cd Desktop/LANL/AFRL-DISC Data/Blackbox brief for LANL

C:\Users\khenk\Desktop\LANL\AFRL-DISC Data\Blackbox brief for LANL


In [3]:
sPCA= pd.read_csv('sPCA_32.csv', sep=',', encoding = 'utf8')
data = sPCA.values
#Load sPCA Dictionary from found sPCA components
mat = sio.loadmat('sPCA_Phi.mat')
sPCA_Phi = mat['sPCA_Phi']

mat = sio.loadmat('fMnist_ytrainlabels.mat')
y_train = np.squeeze(np.asarray(mat['y_train'].T))
mat = sio.loadmat('fMnist_ytestlabels.mat')
y_test = np.squeeze(np.asarray(mat['y_test'].T))


In [7]:
Reconstructed_Data = np.dot(data,sPCA_Phi)
Reconstructed_Data = Reconstructed_Data.reshape(len(Reconstructed_Data),28,28)

In [8]:
#Cutting up image
def get_cell(img, x, y):
    return img[x:x+span,y:y+span].tolist()

In [9]:
#Patch up data
full_set = []
row_size = len(Reconstructed_Data[1])
col_size = len(Reconstructed_Data[2])

span = 7



row_size_list = 4
col_size_list = 4

for i in range(len(Reconstructed_Data)):
    cropped_image = Reconstructed_Data[i]
    
    #normailze image with mean 0 and unit variance
    cropped_image = (cropped_image - np.mean(cropped_image)) / (np.std(cropped_image))
   
    submatrix_list = [get_cell(cropped_image,x,y) for x in range(0,row_size,span) for y in range(0,col_size,span)]
    
    
    d2_list = [submatrix_list[i:i+row_size_list] for i in range(0,len(submatrix_list),row_size_list)]
    #Validation of chopping
    X = np.zeros((row_size,col_size))
    for i in range(row_size_list):
        for j in range(col_size_list):
            for k in range(span):
                for l in range(span):
                    X[i*span+k,j*span+l] = d2_list[i][j][k][l]
    full_set.append(d2_list)
    

Data_Patches = []
for i in range(len(Reconstructed_Data)):
    for j in range(row_size_list):
        for k in range(col_size_list):
            Sample_image_patch = full_set[i][j][k]
            Data_Patches.append(Sample_image_patch)

In [10]:
Data_Patches = np.asarray(Data_Patches)
Data_Patches = Data_Patches.reshape(len(Data_Patches),len(Data_Patches[1])*len(Data_Patches[2]))

In [11]:
#normalize data
for i in range(len(Data_Patches)):
    if LA.norm(Data_Patches[i])!=0:
        Data_Patches[i]=Data_Patches[i]/LA.norm(Data_Patches[i])
# Data_Patches= np.asarray(Data_Patches)
Data_Patches.shape

(1120000L, 49L)

In [12]:
#Creating Randomly imprinted Phi Matrix with number of features = # Neurons(qubits)
Neurons = 64
RandomTrainingSample_Index = random.sample(range(len(Data_Patches)),Neurons)
Rand_Features = Data_Patches[RandomTrainingSample_Index,:]
Rand_Features.shape

(64L, 49L)

In [22]:
#Get Features for data phi and Normalize
Phi = Rand_Features.T
for i in range(Neurons):
    Phi[:,i]= Phi[:,i]/(LA.norm(Phi[:,i]))

In [23]:
def sparse_loss(I,Phi,sparse_representation_coefficients,lam):
    Energy = .5*LA.norm(I-np.dot(Phi,sparse_representation_coefficients))**2 +lam*np.count_nonzero(sparse_representation_coefficients)
    return Energy

def reconstruction_loss(I,Phi,sparse_representation_coefficients):
    Loss = LA.norm(I-np.dot(Phi,sparse_representation_coefficients))**2
    return Loss

In [24]:
#SGD Loop to update Dictionary
Data_Patches = Data_Patches
mini_batch_size = 512
tol = 10**(-32)
max_iterations = 100000
eta =.0001 #Learning Rate (Lower is better)
lam = .35 #found through orthogonal feature reconstructions above
lam_vector = np.full((len(Phi.T), 1), lam, dtype=np.float)
count = 0
Previous_Gradient = 0
momentum_term = .9



Total_Energies = []
Total_Recon_Loss = []
Non_Zero_Coefficients =[] #Helps see how sparse reconstructions are while learning
eta_initial =.01
eta_final = .000001#Learning Rate (Lower is better)
eta = eta_initial
for iteration in range(max_iterations):
    
        
    #Random Draw from Data
    Indexing = random.sample(range(len(Data_Patches)), mini_batch_size)
    
    Neuron_Activation =[]
    Gradients =[]
    Unembedded_Solutions_Training= []
    t1 = time.time()
    New_Solutions = []
    Temp_Data_Patches=[]
    Q = .5*np.dot(np.squeeze(Phi.T),np.squeeze(Phi))
    


    image_number = 0
    Sparse_Rep=[]
    Loss = []
    Sparsity_Loss = []
    Gradients = []
    for sample in Indexing:
        temp = orthogonal_mp(Phi,Data_Patches[sample])
        Sparsity_Loss.append((sparse_loss(Data_Patches[sample],Phi,temp.T,lam)))
        Sparse_Rep.append(temp)
        Loss.append(LA.norm(Data_Patches[sample]-np.dot(Phi,orthogonal_mp(Phi,Data_Patches[sample]))))
        error = np.subtract(Data_Patches[sample].T,np.squeeze(np.dot(Phi,temp.T)))
        Delta_Phi =np.outer(error,temp)
        Delta_Phi = np.asarray(Delta_Phi)
        Gradients.append(Delta_Phi)
        
    Gradient = sum(np.asarray(Gradients))/mini_batch_size
    #print(Gradient.shape)
    #Phi_New = Phi - (eta/mini_batch_size)*np.squeeze(Delta_Phi).T + momentum_term*Previous_Gradient
    Phi_New = Phi - (eta)*Gradient + momentum_term*Previous_Gradient #non division by mini batch size
    Previous_Gradient = Gradient
    
    for i in range(Neurons):
        Phi_New[:,i]= Phi_New[:,i]/(math.sqrt(np.dot(Phi_New[:,i].T,Phi_New[:,i])))
    Previous_Gradient = Gradient 

    Loss = sum(np.asarray(Loss))/len(Indexing)
    Sparsity_Loss = sum(np.asarray(Sparsity_Loss))/len(Indexing)
    Total_Energies.append(Sparsity_Loss)
    Total_Recon_Loss.append(Loss)

#
    
    if count%10==0:
        print(count)
        print(Loss,Sparsity_Loss,np.count_nonzero(Sparse_Rep))
        print(LA.norm(Phi_New-Phi))
    #print(plt.plot(range(iteration+1), Total_Energies ))
    #print('Energies,Epoc :', Total_Energies,count)
    elapsed1 = time.time() - t1
    #print(elapsed1)
    count += 1

#     sio.savemat('Phi_New_Fashion_MiniBatch1000_sPCA_AFRL_32_Lambda_'+str(lam)+'_OMP.mat', {'Phi_New':Phi_New})
#     sio.savemat('TotalEnergies_Fashion_MiniBatch1000_sPCA_AFRL_32_Lambda_'+str(lam)+'_OMP.mat', {'Total_Energies':Total_Energies})
    if LA.norm(Phi_New-Phi) > tol:
        #print(LA.norm(Phi_New-Phi))
        Phi = Phi_New
        eta = eta_initial*((eta_final/float(eta_initial))**(iteration/float(max_iterations))) #exponentially decaying learing rate


    else:
        print('converged')
        
        break


0
(0.18648182834070873, 2.1213160814507845, 3072)
0.00016526735144625676
10
(0.17843195788054741, 2.119418984691764, 3072)
0.013623511554889374
20
(0.17530786722895975, 2.1189968139579825, 3072)
0.012815319606128575
30
(0.18409986089071428, 2.1206878798775426, 3072)
0.012079983802246503
40
(0.17667085439269517, 2.1187734210265954, 3072)
0.01112413310434853
50
(0.16983347273958532, 2.1176826627545684, 3072)
0.009421909837773782
60
(0.16752051821446298, 2.1167566987393047, 3072)
0.009650641042741832
70
(0.16182318426038925, 2.115893227146937, 3072)
0.010395530865382632
80
(0.1679309217823369, 2.1171760885150044, 3072)
0.00887819583421217
90
(0.1631122852449611, 2.115962625812599, 3072)
0.009169395790206328
100
(0.17179096003383937, 2.117979122246138, 3072)
0.008868057627749195
110
(0.16380215766122666, 2.1161334498729474, 3072)
0.00882034946007104
120
(0.1623619418248953, 2.11571835525813, 3072)
0.009002547085921101
130
(0.15670910379804545, 2.1149303610754764, 3072)
0.008677199472654742

1140
(0.14750876493986018, 2.1129500606936125, 3072)
0.006816484131481356
1150
(0.14537362122740202, 2.1125129413261488, 3072)
0.006500889685403826
1160
(0.14525752437396314, 2.112528416533209, 3072)
0.006700160202267791
1170
(0.14044855346633087, 2.111634170941442, 3072)
0.006287023533731319
1180
(0.14124732103548016, 2.1118075284637072, 3072)
0.006737229726895583
1190
(0.14587071243589714, 2.112660841139496, 3072)
0.006654853114742919
1200
(0.14407348181871754, 2.1123164379591253, 3072)
0.0066942124744731055
1210
(0.14158391357514705, 2.1117768601821107, 3072)
0.006657652952818937
1220
(0.14178901026361024, 2.111895406804228, 3072)
0.0069726256012776055
1230
(0.141592608283927, 2.1122282147750284, 3072)
0.006769716239593839
1240
(0.15005706225891463, 2.113739224691841, 3072)
0.006044816531289642
1250
(0.14302844043891305, 2.112191723373272, 3072)
0.007212987428752396
1260
(0.14044009248554482, 2.11154345067698, 3072)
0.006666851487564397
1270
(0.14414315314197304, 2.1123807785289515,

2260
(0.14242456980972795, 2.112110833351904, 3072)
0.006260821291911576
2270
(0.1337806574528178, 2.110758660808814, 3072)
0.006413739818930607
2280
(0.14374252496086057, 2.112349590969349, 3072)
0.006755293978030771
2290
(0.13958612604240833, 2.111593296633526, 3072)
0.006535755549647624
2300
(0.14265808030001673, 2.1119748142625276, 3072)
0.006044513924011124
2310
(0.14188269799056824, 2.1121935691357003, 3072)
0.0065086024578187435
2320
(0.14098967833108225, 2.1117862432841923, 3072)
0.0066090268342686815
2330
(0.1427512957099506, 2.1120960225525773, 3072)
0.0066261277101904055
2340
(0.1434100947149785, 2.1123850524891155, 3072)
0.00643642476830483
2350
(0.13879990804523593, 2.1114522771014617, 3072)
0.006407721491144249
2360
(0.14524496543217402, 2.112505970012674, 3072)
0.006672134740597041
2370
(0.13811637811227065, 2.1112111229619286, 3072)
0.006465231087943551
2380
(0.1394877767829663, 2.1116222053511393, 3072)
0.0063916830788963135
2390
(0.14100775593702344, 2.111697019279701

KeyboardInterrupt: 